In [ ]:
%pip install -qU langchain-openai

In [ ]:
%pip install langchain

In [ ]:
import langchain

# LLM

In [ ]:
from langchain_openai import OpenAI

In [ ]:
llm = OpenAI()

In [ ]:
llm('A cat climbed the wall,')

In [ ]:
llm.invoke('A cat climbed the wall,')

In [ ]:
result = llm.generate(['A cat climbed the wall,', 'Jack and Jill'])

In [ ]:
result.schema()

In [ ]:
result.llm_output

In [ ]:
result.generations

In [ ]:
print(result.generations[0][0].text)

# Chat Models

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chat = ChatOpenAI()

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
result = chat([SystemMessage(content='You are a Bank employee'), HumanMessage(content='Tell me a joke about your life')])

In [ ]:
print(result.content)

In [ ]:
result = chat.generate([
    [SystemMessage(content='You are a lazy Bank employee, who hates your job'),
     HumanMessage(content='Tell me a joke about your life')],
    [SystemMessage(content='You are a very Professional Bank employee, who loves your job'),
     HumanMessage(content='Tell me a joke about your life')]
    ])

In [ ]:
result.llm_output

In [ ]:
result.generations

In [ ]:
print(result.generations[0][0].text)

In [ ]:
print(result.generations[1][0].text)

In [ ]:
result = chat(
    [SystemMessage(content='You are a lazy Bank employee, who hates your job'),
     HumanMessage(content='Tell me a joke about your bank life')],
     temperature=0, max_tokens=20
    )

In [ ]:
print(result.content)

In [ ]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [ ]:
llm.predict('Tell me a fact about Mars')

In [ ]:
llm.predict('Tell me a fact about Mars')

# Prompt Templates

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
no_input_prompt = PromptTemplate(input_variables=[],
                                template = 'Tell me a fact')

In [ ]:
print(llm(no_input_prompt.format()))

In [ ]:
single_input_prompt = PromptTemplate(input_variables=['topic'],
                                    template='Tell me a fact about {topic}')

In [ ]:
single_input_prompt.format(topic= 'Pluto')

In [ ]:
print(llm(single_input_prompt.format(topic= 'Pluto')))

# Chat Prompt Template

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [ ]:
system_template = "You are an AI recipe Assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}"

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
human_template = "{recipe_request}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
system_message_prompt.input_variables

In [ ]:
human_message_prompt.input_variables

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chat_prompt.input_variables

In [ ]:
prompt = chat_prompt.format_prompt(cooking_time='60 min', dietary_preference='Vegan', recipe_request='Quick Snack').to_messages()

In [ ]:
result = chat(prompt)

In [ ]:
print(result.content)

# Few Shot Prompts

In [ ]:
system_template = "You are an AI recipe Assistant that specializes in giving the wacky definitions for acronyms"

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
# few shot examples
acronyms = "A.I.R"
example_input_one = HumanMessagePromptTemplate.from_template(acronyms)

wacky_abbrevations = "Absolute Insane Rabbit"
example_output_one = HumanMessagePromptTemplate.from_template(wacky_abbrevations)

In [ ]:
human_template = "{new_acronym}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
Chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_input_one, example_output_one, human_message_prompt])

In [ ]:
Chat_prompt.input_variables

In [ ]:
request = Chat_prompt.format_prompt(new_acronym="P.E.T").to_messages()

In [ ]:
request

In [ ]:
few_shot_result = chat(request)

In [ ]:
print(few_shot_result.content)

# Parsing outputs

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
parser = CommaSeparatedListOutputParser()

In [ ]:
parser.get_format_instructions()

In [ ]:
reply = "red, blue, green"

In [ ]:
parser.parse(reply)

In [ ]:
human_template = '{request}\n{format_instructions}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
human_message_prompt.input_variables

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
request = chat_prompt.format_prompt(request='Give me five breeds of dogs', format_instructions= parser.get_format_instructions()).to_messages()

In [ ]:
request

In [ ]:
response = chat(request)

In [ ]:
response

In [ ]:
print(response.content)

In [ ]:
print(parser.parse(response.content))

# Document Loaders

In [ ]:
from langchain_community.document_loaders import HNLoader

In [ ]:
loader = HNLoader("https://news.ycombinator.com/item?id=34817881")

In [ ]:
data = loader.load()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chat = ChatOpenAI()

In [ ]:
print(data[0].page_content)

In [ ]:
human_template = "Give me a short summary on the following Hackernews Community comment: \n{comment}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
chat_prompt.input_variables

In [ ]:
prompt = chat_prompt.format_prompt(comment=data[0].page_content).to_messages()

In [ ]:
response = chat(prompt)

In [ ]:
print(response.content)